In [2]:
pip install -r requirements.txt



  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.11
    Uninstalling pydantic-1.10.11:
      Successfully uninstalled pydantic-1.10.11



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from pypdf import PdfReader
import re

def load_pdf(file_path):
    """
    Reads the text content from a PDF file and returns it as a single string.

    Parameters:
    - file_path (str): The file path to the PDF file.

    Returns:
    - str: The concatenated text content of all pages in the PDF.
    """
    reader = PdfReader(file_path)
    
    text = ""
    for page in reader.pages:
        text += page.extract_text()

    return text

pdf_text = load_pdf(file_path="constituion-pdf.pdf")

In [4]:
def save_text_to_file(text, output_file_path):
    """
    Saves the given text to a text file.

    Parameters:
    - text (str): The text content to save.
    - output_file_path (str): The file path where the text will be saved.
    """
    with open(output_file_path, "w", encoding="utf-8") as file:
        file.write(text)

In [5]:
save_text_to_file(text=pdf_text, output_file_path="output.txt")

In [6]:
from hide import hf_api
inference_api_key = hf_api

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

loader = TextLoader("output.txt", encoding="utf-8")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300, length_function=len)
docs = text_splitter.split_documents(documents)
print(len(docs))
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=inference_api_key, model_name="sentence-transformers/all-MiniLM-l6-v2"
)
db = FAISS.from_documents(docs, embeddings)
print(db.index.ntotal)

db.save_local("faiss_index_law")

1227
1227
